In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
import pandas as pd

In [ ]:
df = pd.read_csv("Taxi_fare.csv")

In [ ]:
#Checking whether there is any Null value in the DataFrame
df.isna().sum()

trip_duration         0
distance_traveled     0
num_of_passengers     0
fare                  0
tip                   0
miscellaneous_fees    0
total_fare            0
surge_applied         0
dtype: int64

In [ ]:
#Moving the target col to the Last
target_col = df.pop("total_fare")
df["total_fare"] = target_col

In [ ]:
#Extracting features and Labels from the Dataset
features = df.drop('total_fare', axis=1).values
labels = df['total_fare'].values

#Creating class CustomDataset

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,data,label):
    self.data = data
    self.label = label
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    samples = {"data" : torch.tensor(self.data[idx] , dtype = torch.float32),
               "label" : torch.tensor(self.label[idx],dtype = torch.float32)}
    return samples

In [ ]:
#Creating the instance of the class
customdataset = CustomDataset(features,labels)

In [ ]:
#Finding the input and target size inorder to give it to the model
input_size = features.shape[1]
target_size = 1
print(f"Size of the input is {input_size} , target size is {target_size}")

Size of the input is 7 , target size is 1


#Creating the model

In [ ]:
#Creating the reproducability
torch.manual_seed(42)
#model
model = nn.Linear(input_size,target_size)

In [ ]:
model.state_dict()

OrderedDict([('weight',
              tensor([[ 0.2890,  0.3137, -0.0885,  0.3472, -0.0828,  0.0763, -0.1840]])),
             ('bias', tensor([0.2220]))])

Creating the Optimizer and Loss function

In [ ]:
learning_rate = 0.001
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [ ]:
batch_size = 100
num_workers = 4
data_loader = DataLoader(customdataset,batch_size = batch_size, num_workers = 10 ,shuffle=True)

In [ ]:
torch.manual_seed(42)
epoch = 100
#1
for i in range(epoch):
  for samples in data_loader:
    data,label = samples["data"] , samples["label"]
    model.train()
    #ForwardPass
    y_pred = model(data)

    #Loss
    loss = loss_fn(y_pred,label)

    #Optimizer
    optimizer.zero_grad()

    #Loss backward
    loss.backward()

    #Update weights
    optimizer.step()


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([73])) that is different to the input size (torch.Size([73, 1])

In [ ]:
loss

tensor(424.6856, grad_fn=<MeanBackward0>)